In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import cpca
import os
os.chdir("D:\Git Local\CloudSeeding\code\灾害数据")
df = pd.read_stata("自然灾害事件_since_2014_CN.dta")

# clean the data
df.drop([0,1],axis=0,inplace=True)
df['City'] = df['City'].str.replace("、",",")
df['City'] = df['City'].str.replace("，",",")
df['City'] = df['City'].str.split(',')
df = df.explode('City')

# find the adcode
df.reset_index(drop=True,inplace=True)
df['adcode'] = cpca.transform(df["City"])['adcode']
df2 = df[df['adcode'].isna()]

df2.reset_index(drop=True,inplace=True)
df2['adcode'] = cpca.transform(df["Areas"])['adcode']

data = pd.concat([df,df2])
data.dropna(subset=['adcode'],inplace=True)

# Drop rows where last four digits of adcode are "0000", except for specified values
mask = (data['adcode'].astype(str).str[-4:] == "0000") & (~data['adcode'].isin([120000, 110000, 500000, 310000]))
data = data[~mask]

# Generate citycode2 from the first four digits of adcode
data['citycode2'] = data['adcode'].astype(str).str[:4]
data['citycode2'] = data['citycode2'].astype(int)

# Adjust citycode2 for direct-controlled municipalities
for code, new_code in zip(["11", "12", "31", "50"], [1100, 1200, 3100, 5000]):
    data.loc[data['citycode2'].astype(str).str[:2] == code, 'citycode2'] = new_code

data.to_csv("disaster_adcode.csv")

C:\Users\Anora\AppData\Local\Temp\ipykernel_4576\2500311126.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['adcode'] = cpca.transform(df["Areas"])['adcode']
